# **Downloading The dataset**

you need to download the images from the dataset,The excel file provided in the labeled dataset contains the images paths and thier labels. make sure that the images are downloaded into the following directory (/content/Data)

# **Data preprocessing and augmentation**

The excel file provided in the labeled dataset contains the images paths and thier labels.

In [ ]:
cd /content/Data

In [ ]:
import os
import pandas as pd
from shutil import copyfile

def organize_images_by_label(excel_path, image_column, label_column, output_dir):
    # Read the Excel file
    data = pd.read_excel(excel_path)

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate through rows and copy images to label-specific folders
    for index, row in data.iterrows():
        try:
            image_path = str(row[image_column])  # Convert to string
            label = str(row[label_column])  # Convert to string

            # Skip rows where image path or label is not a valid string
            if not image_path.strip() or not label.strip():
                print(f"Skipping row {index + 2} due to empty image path or label.")
                continue

            # Create label folder if it doesn't exist
            label_folder = os.path.join(output_dir, label)
            if not os.path.exists(label_folder):
                os.makedirs(label_folder)

            # Copy image to label folder
            _, image_filename = os.path.split(image_path)
            destination_path = os.path.join(label_folder, image_filename)

            # Check if destination image already exists, and if not, copy
            if not os.path.exists(destination_path):
                copyfile(image_path, destination_path)
            else:
                print(f"Warning: Image '{image_filename}' already exists in '{label}' folder.")
        except Exception as e:
            print(f"Error processing row {index + 2}: {e}")

if __name__ == "__main__":
    # Replace these with your actual file paths and column names
    excel_file_path = '/content/Labels.xlsx'
    image_column_name = 'image_path'
    label_column_name = 'Best_DET'
    output_directory = '/content/new_data/data'

    organize_images_by_label(excel_file_path, image_column_name, label_column_name, output_directory)


In [ ]:
import os

def count_images_in_folders(root_folder):
    total_images = 0

    # Iterate through subfolders
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Count images in the subfolder
            images_in_folder = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))])

            print(f"Folder '{folder_name}': {images_in_folder} images")

            # Update the total count
            total_images += images_in_folder

    print(f"\nTotal number of images in all folders: {total_images}")

if __name__ == "__main__":
    # Replace with the path to your root folder
    root_folder_path = '/content/new_data/data'

    count_images_in_folders(root_folder_path)


**The following cells should be run 8 times, one time for each class to generate more data samples. just change the name of the detector in the path.**

for example:
change ("/content/new_data/data/FTRCNN") to ("/content/new_data/data/DETR") and so on for each class.

In [ ]:
pip install Augmentor

In [ ]:
import os
from PIL import Image

def count_images_in_folder(folder_path):
    # Ensure the folder path exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' not found.")
        return

    # Get a list of all files in the folder
    all_files = os.listdir(folder_path)

    # Filter only the image files (assuming JPEG format for simplicity)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg'))]

    # Count the number of image files
    num_images = len(image_files)

    print(f"Number of images in folder '{folder_path}': {num_images}")

# Example usage:
folder_path = '/content/new_data/data/FTRCNN'
count_images_in_folder(folder_path)


In [ ]:
import Augmentor
p = Augmentor.Pipeline("/content/new_data/data/FTRCNN")

In [ ]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)

In [ ]:
p.sample(2000)

In [ ]:
import os
import shutil

def move_images(source_folder, destination_folder):
    # Ensure both source and destination folders exist
    if not os.path.exists(source_folder):
        print(f"Error: Source folder '{source_folder}' not found.")
        return

    if not os.path.exists(destination_folder):
        print(f"Error: Destination folder '{destination_folder}' not found.")
        return

    # Get a list of all files in the source folder
    all_files = os.listdir(source_folder)

    # Filter only the image files (assuming JPEG format for simplicity)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg','.png'))]

    # Move each image file to the destination folder
    for image_file in image_files:
        source_path = os.path.join(source_folder, image_file)
        destination_path = os.path.join(destination_folder, image_file)

        # Move the file
        shutil.move(source_path, destination_path)

        print(f"Moved: {image_file}")

    print("Image move operation completed.")

# Example usage:
source_folder = '/content/new_data/data/FTRCNN/output'
destination_folder = '/content/new_data/data/FTRCNN'

move_images(source_folder, destination_folder)


In [ ]:
import os
from PIL import Image

def count_images_in_folder(folder_path):
    # Ensure the folder path exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' not found.")
        return

    # Get a list of all files in the folder
    all_files = os.listdir(folder_path)

    # Filter only the image files (assuming JPEG format for simplicity)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg','png'))]

    # Count the number of image files
    num_images = len(image_files)

    print(f"Number of images in folder '{folder_path}': {num_images}")

# Example usage:
folder_path = '/content/new_data/data/FTRCNN'
count_images_in_folder(folder_path)


**The following cells will split the data into training and testing splits to prepare for the training**

In [ ]:
import os
from sklearn.model_selection import train_test_split
from shutil import copyfile

def split_dataset(input_folder, output_train_folder, output_test_folder, test_size=0.2, random_seed=42):
    # Get the list of class folders in the input folder
    class_folders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))]

    # Create output folders if they don't exist
    os.makedirs(output_train_folder, exist_ok=True)
    os.makedirs(output_test_folder, exist_ok=True)

    # Loop through each class folder
    for class_folder in class_folders:
        class_path = os.path.join(input_folder, class_folder)

        # Get the list of image files for the current class
        image_files = [f for f in os.listdir(class_path) if f.endswith('.jpg') or f.endswith('.png')]

        # Check if there are enough samples for splitting
        if len(image_files) < 2:
            print(f"Skipping class '{class_folder}' due to insufficient samples.")
            continue

        # Split the image files into training and testing sets
        train_files, test_files = train_test_split(image_files, test_size=test_size, random_state=random_seed)

        # Copy training images to the output training folder
        for train_file in train_files:
            input_path = os.path.join(class_path, train_file)
            output_path = os.path.join(output_train_folder, class_folder, train_file)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            copyfile(input_path, output_path)

        # Copy testing images to the output testing folder
        for test_file in test_files:
            input_path = os.path.join(class_path, test_file)
            output_path = os.path.join(output_test_folder, class_folder, test_file)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            copyfile(input_path, output_path)

    print("Dataset split into training and testing sets.")

# Example usage:
input_folder = '/content/new_data/data'
output_train_folder = '/content/new_data/data/train'
output_test_folder = '/content/new_data/data/valid'

split_dataset(input_folder, output_train_folder, output_test_folder)


In [ ]:
import os

def count_images_in_folders(root_folder):
    total_images = 0

    # Iterate through subfolders
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Count images in the subfolder
            images_in_folder = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))])

            print(f"Folder '{folder_name}': {images_in_folder} images")

            # Update the total count
            total_images += images_in_folder

    print(f"\nTotal number of images in all folders: {total_images}")

if __name__ == "__main__":
    # Replace with the path to your root folder
    root_folder_path = '/content/new_data/data/train'

    count_images_in_folders(root_folder_path)


In [ ]:
import os

def count_images_in_folders(root_folder):
    total_images = 0

    # Iterate through subfolders
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Count images in the subfolder
            images_in_folder = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))])

            print(f"Folder '{folder_name}': {images_in_folder} images")

            # Update the total count
            total_images += images_in_folder

    print(f"\nTotal number of images in all folders: {total_images}")

if __name__ == "__main__":
    # Replace with the path to your root folder
    root_folder_path = '/content/new_data/data/valid'

    count_images_in_folders(root_folder_path)


# **Training ResNet**

In [ ]:
pip install timm

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load data
data_dir = '/content/new_data/data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Load pre-trained ResNet and modify the final layer
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # Adjust the final layer
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
def train_model(model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            epoch_precision = precision_score(all_labels, all_preds, average='weighted')
            epoch_recall = recall_score(all_labels, all_preds, average='weighted')
            epoch_f1 = f1_score(all_labels, all_preds, average='weighted')

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Precision: {epoch_precision:.4f} Recall: {epoch_recall:.4f} F1: {epoch_f1:.4f}')

    return model

# Fine-tune the model
model_ft = train_model(model, criterion, optimizer, num_epochs=15)
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load data
data_dir = '/content/new_data/data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Load pre-trained ResNet and modify the final layer
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # Adjust the final layer
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
def train_model(model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            epoch_precision = precision_score(all_labels, all_preds, average='weighted')
            epoch_recall = recall_score(all_labels, all_preds, average='weighted')
            epoch_f1 = f1_score(all_labels, all_preds, average='weighted')

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Precision: {epoch_precision:.4f} Recall: {epoch_recall:.4f} F1: {epoch_f1:.4f}')

    return model

# Fine-tune the model
model_ft = train_model(model, criterion, optimizer, num_epochs=15)

# Save the fine-tuned model
torch.save(model_ft.state_dict(), 'fine_tuned_resnet50.pth')

# Save the fine-tuned model
torch.save(model_ft.state_dict(), 'fine_tuned_resnet50.pth')


In [ ]:
import shutil
import os

# Define the source file path and the destination directory path
source_file = '/content/Data/resnet_swav_finetuned.pth'
destination_dir = '/content'

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Construct the full destination file path
destination_file = os.path.join(destination_dir, os.path.basename(source_file))

# Move the file
shutil.move(source_file, destination_file)

print(f"File moved from {source_file} to {destination_file}")


# **ResNet(SwAV)**

In [ ]:
# Install necessary libraries
!pip install timm

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import timm

# Define the paths
TRAIN_DATA_PATH = '/content/new_data/data/train'
VALID_DATA_PATH = '/content/new_data/data/valid'

# Define data transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(TRAIN_DATA_PATH, transform=train_transform)
val_dataset = datasets.ImageFolder(VALID_DATA_PATH, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

# Define the ResNet50 model class with SwAV pre-trained weights
class ResNetSwAV(nn.Module):
    def __init__(self, num_classes):
        super(ResNetSwAV, self).__init__()
        # Load ResNet50 model with SwAV pre-trained weights
        self.resnet = torch.hub.load('facebookresearch/swav:main', 'resnet50')
        # Modify the final layer to match the number of classes in your dataset
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Initialize the model
NUM_CLASSES = len(train_dataset.classes)
model = ResNetSwAV(NUM_CLASSES)

# Define device, criterion, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training and validation loop
EPOCHS = 10
best_val_accuracy = 0.0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    train_accuracy = 100. * correct_train / total_train
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss/len(train_loader)}, Train Accuracy: {train_accuracy:.2f}%")

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    val_accuracy = 100. * correct_val / total_val
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'resnet_swav_finetuned.pth')
        print(f'Saved model with validation accuracy: {val_accuracy:.2f}%')

# Plot training and validation performance
import matplotlib.pyplot as plt

# Example plot
train_accuracies = [train_accuracy]  # Replace with actual training accuracy list
val_accuracies = [val_accuracy]  # Replace with actual validation accuracy list

plt.figure(figsize=(10, 5))
plt.plot(range(1, EPOCHS + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, EPOCHS + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Performance')
plt.legend()
plt.show()



In [ ]:
import shutil
import os

# Define the source file path and the destination directory path
source_file = '/content/Data/resnet_swav_finetuned.pth'
destination_dir = '/content'

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Construct the full destination file path
destination_file = os.path.join(destination_dir, os.path.basename(source_file))

# Move the file
shutil.move(source_file, destination_file)

print(f"File moved from {source_file} to {destination_file}")


# ResNet(**BT**)

In [ ]:
pip install timm

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models.resnet import Bottleneck, ResNet

# Define the custom ResNetTrunk class and functions as provided
class ResNetTrunk(ResNet):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        del self.fc  # remove FC layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x


def get_pretrained_url(key):
    URL_PREFIX = "https://github.com/lunit-io/benchmark-ssl-pathology/releases/download/pretrained-weights"
    model_zoo_registry = {
        "BT": "bt_rn50_ep200.torch",
        "MoCoV2": "mocov2_rn50_ep200.torch",
        "SwAV": "swav_rn50_ep200.torch",
    }
    pretrained_url = f"{URL_PREFIX}/{model_zoo_registry.get(key)}"
    return pretrained_url


def resnet50(pretrained, progress, key, **kwargs):
    model = ResNetTrunk(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        pretrained_url = get_pretrained_url(key)
        verbose = model.load_state_dict(
            torch.hub.load_state_dict_from_url(pretrained_url, progress=progress)
        )
        print(verbose)
    return model

# Define the paths
TRAIN_DATA_PATH = '/content/new_data/data/train'
VALID_DATA_PATH = '/content/new_data/data/valid'

# Define data transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(TRAIN_DATA_PATH, transform=train_transform)
val_dataset = datasets.ImageFolder(VALID_DATA_PATH, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

# Define the complete model with a classification head
class ResNetBTFineTune(nn.Module):
    def __init__(self, num_classes):
        super(ResNetBTFineTune, self).__init__()
        self.trunk = resnet50(pretrained=True, progress=False, key="BT")
        self.fc = nn.Linear(2048, num_classes)  # Assuming the output from trunk has 2048 features

    def forward(self, x):
        x = self.trunk(x)
        x = nn.AdaptiveAvgPool2d((1, 1))(x)  # Global Average Pooling
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize the model
NUM_CLASSES = len(train_dataset.classes)
model = ResNetBTFineTune(NUM_CLASSES)

# Define device, criterion, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training and validation loop
EPOCHS = 15
best_val_accuracy = 0.0
train_accuracies = []
val_accuracies = []

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    train_accuracy = 100. * correct_train / total_train
    train_accuracies.append(train_accuracy)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss/len(train_loader)}, Train Accuracy: {train_accuracy:.2f}%")

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    val_accuracy = 100. * correct_val / total_val
    val_accuracies.append(val_accuracy)
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'resnet_bt_finetuned.pth')
        print(f'Saved model with validation accuracy: {val_accuracy:.2f}%')

# Plot training and validation performance
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, EPOCHS + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, EPOCHS + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Performance')
plt.legend()
plt.show()


In [ ]:
import shutil
import os

# Define the source file path and the destination directory path
source_file = '/content/Data/resnet_bt_finetuned.pth'
destination_dir = '/content'

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Construct the full destination file path
destination_file = os.path.join(destination_dir, os.path.basename(source_file))

# Move the file
shutil.move(source_file, destination_file)

print(f"File moved from {source_file} to {destination_file}")
